# scrapy project
- 프로젝트 생성
- 구조
- gmarket 베스트 상품 링크, 상세정보 수집

## 프로젝트 생성

In [2]:
!scrapy startproject crawler

New Scrapy project 'crawler', using template directory 'c:\users\kgmyh\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    E:\OneDrive - 둔전초등학교\01_과목별 수업자료\03_크롤링\03_차후 수업 추가할 것\scarappy_exam\crawler

You can start your first spider with:
    cd crawler
    scrapy genspider example example.com


In [6]:
!tree crawler

UX200P 볼륨에 대한 폴더 경로의 목록입니다.
볼륨 일련 번호는 1858-08E9입니다.
E:\ONEDRIVE - 둔전초등학교\01_과목별 수업자료\03_크롤링\03_차후 수업 추가할 것\SCARAPPY_EXAM\CRAWLER
└─crawler
    ├─spiders
    │  └─__pycache__
    └─__pycache__


### 구조
- spiders 디렉토리
    - 어떤 웹서비스를 어떻게 크롤링 할 것인지를 작성 (.py로 작성)
- items.py
    - 어떤 데이터를 저장할 지 Model에 해당. 저장하는 데이터의 구조를 설정
- pipelines.py
    - 스크래핑한 결과물을 item 형태로 구성하고 처리하는 방법에 대한 코드
    - 처리한 다는 것은 DB에 저장할지 메일로 보낼지등에 대한 처리방법
- middlewares.py
    - HW 관련 설정. 이건 그대로 둔다.
- settings.py
    - 스크래핑 할 때의 환경설정값을 지정.
    - robots.txt 를 따를지 안따를지 등

### 절차
- spiders/xxxx.py -> 크롤링 절차를 작성한다. (ex: gmarket.py)
- 가져온 데이터는 items.py에 정의한 스키마에 따라 item으로 만들어 진다.
- 이것을 던지면 출력되는데 pipelines.py에 정의한 것이 있으면 여기를 거쳐 처리가 된뒤 출력된다.
- 내부적으로 Thread로 처리된다.

### 구현
- spiders\gmarkets.py 에 구현
- items.py 구현

### gmarket 베스트셀러 상품수집
- 상품명, 상세페이지 URL, 원가, 판매가, 할인률
- xpath 확인
- items.py 변경
- spider.py 구현
- 크롤러 실행

#### 1. xpath 확인
- http://corners.gmarket.co.kr/Bestsellers

In [9]:
import scrapy
import requests
from scrapy.http import TextResponse

In [10]:
url = 'http://corners.gmarket.co.kr/Bestsellers'

In [11]:
req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding='utf-8')

In [17]:
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li/div[1]/a/@href').extract()
len(links)

200

In [18]:
links[0]

'http://item.gmarket.co.kr/Item?goodscode=884451980&ver=637304448090964285'

In [31]:
req = requests.get(links[0])
response = TextResponse(req.url, body=req.text, encoding='utf-8')
title = response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract()
sales_price = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract().replace(',','')
orginal_price = response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract().replace(',','')
discount_rate = str(round(1-int(sales_price)/int(orginal_price), 2)*100) + '%' # 1-정가에 대한 판매가의 비율  (정가가 100 판매가가 40 100-40=60원 할인 할인된 60원은 정가 대비 비율이? 60/100 => 이걸 1-세일가/판매가)
print(title, sales_price, orginal_price, discount_rate)

[구운감자] 해태 구운감자 24g 30개 과자 간식 한박스 오늘만 할인  14900 37300 60.0%


In [32]:
# items.py 작성

In [41]:
!type .\crawler\crawler\items.py
# 경로 \ 로 해야 함. / 안됨(윈도우경로)

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class CrawlerItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass


In [37]:
import os
print(os.getcwd())

E:\OneDrive - 둔전초등학교\01_과목별 수업자료\03_크롤링\03_차후 수업 추가할 것\scarappy_exam


In [61]:
%%writefile crawler/crawler/items.py
# 셀의 내용을 파일에 출력(덮어쓰기) 매직 커맨드는 첫줄에
import scrapy

# scrapy.Item 상속해서 만든다. 클래스 이름은 상관없다.
class CrawlerItem(scrapy.Item):
    title = scrapy.Field()
    sales_price = scrapy.Field()
    original_price = scrapy.Field()
    discount_rate = scrapy.Field()
    link  = scrapy.Field()
    

Overwriting crawler/crawler/items.py


In [69]:
%%writefile crawler/crawler/spiders/spider.py
import scrapy
from crawler.items import CrawlerItem  #item import

class Spider(scrapy.Spider):
    name = "GmarketBestSellers" #실행시킬 이름
    allow_domain = ['gmarket.co.kr'] # 이 도메인 내의 내용만 크롤링 하겠다.
    start_urls = ['http://corners.gmarket.co.kr/Bestsellers'] #최초에 크롤링할 페이지의 url. 여러개 등록 가능. 여러 url이면 동시에 여러개가 실행된다.
    
    # request에 대한 response를 받아 이 메소드가 실행된다.
    def parse(self, response):
        # 목록의 링크들 가져와서 반복문 돌린다.
        links = links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li/div[1]/a/@href').extract()
        # 하나의 링크를 가져와 요청한뒤 그 response를 callback 에게 전달한다.
        for link in links[:10]:
            yield scrapy.Request(link, callback=self.page_content)
    
    # 상세페이지의 response를 받아서 원하는 처리를 하면된다.
    def page_content(self, response):
        item = CrawlerItem()
        item['title'] =  response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract()
        item['sales_price'] = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract().replace(',','')
        try:
        # 오리지날 가격이 없는 경우 element가 없어서 예외 발생한다. 이런 경우 sales_price를 넣는다.
            item['original_price'] = response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract().replace(',','')
        except:
            item['original_price'] = item['sales_price']
        item['discount_rate'] = str(round(1-int(item['sales_price'])/int(item['original_price']), 2)*100) + '%'
        item['link'] = response.url # 요청한 url을 받을 수 있다. 
        yield item

Overwriting crawler/crawler/spiders/spider.py


### scrapy 실행

In [52]:
!dir crawler
# scrapy.cfg 파일이 있는 디렉토리에서 실행한다.

 E 드라이브의 볼륨: UX200P
 볼륨 일련 번호: 1858-08E9

 E:\OneDrive - 둔전초등학교\01_과목별 수업자료\03_크롤링\03_차후 수업 추가할 것\scarappy_exam\crawler 디렉터리

2020-07-15  오후 09:06    <DIR>          .
2020-07-15  오후 09:06    <DIR>          ..
2020-07-15  오후 09:06    <DIR>          crawler
2020-07-15  오후 09:06               257 scrapy.cfg
               1개 파일                 257 바이트
               3개 디렉터리  144,591,044,608 바이트 남음


In [55]:
%%writefile run.bat

cd crawler
scrapy crawl GmarketBestSellers

Overwriting run.bat


### 실행

In [70]:
!run.bat


(base) E:\OneDrive - 둔전초등학교\01_과목별 수업자료\03_크롤링\03_차후 수업 추가할 것\scarappy_exam>cd crawler 

(base) E:\OneDrive - 둔전초등학교\01_과목별 수업자료\03_크롤링\03_차후 수업 추가할 것\scarappy_exam\crawler>scrapy crawl GmarketBestSellers 


2020-07-15 22:10:19 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: crawler)
2020-07-15 22:10:19 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Windows-10-10.0.18362-SP0
2020-07-15 22:10:19 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-07-15 22:10:19 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'crawler',
 'NEWSPIDER_MODULE': 'crawler.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['crawler.spiders']}
2020-07-15 22:10:19 [scrapy.extensions.telnet] INFO: Telnet Password: b0744890281b08f2
2020-07-15 22:10:19 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-07-15 22:10:19

In [73]:
# 파일 cell 로 불러오는 매직커맨드
# 불러오고 매직커맨드 명령어는 주석처리된다.

In [ ]:
%load crawler/crawler/spiders/spider.py 

### 결과를 csv로 저장하도록 처리

In [76]:
%%writefile run.bat

cd crawler
scrapy crawl GmarketBestSellers -o GmarketBestSellers.csv


Overwriting run.bat


In [77]:
!run.bat


(base) E:\OneDrive - 둔전초등학교\01_과목별 수업자료\03_크롤링\03_차후 수업 추가할 것\scarappy_exam>cd crawler 

(base) E:\OneDrive - 둔전초등학교\01_과목별 수업자료\03_크롤링\03_차후 수업 추가할 것\scarappy_exam\crawler>scrapy crawl GmarketBestSellers -o GmarketBestSellers.csv 


2020-07-15 22:32:46 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: crawler)
2020-07-15 22:32:46 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Windows-10-10.0.18362-SP0
2020-07-15 22:32:46 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-07-15 22:32:46 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'crawler',
 'NEWSPIDER_MODULE': 'crawler.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['crawler.spiders']}
2020-07-15 22:32:46 [scrapy.extensions.telnet] INFO: Telnet Password: 8da9d2e6f926145c
2020-07-15 22:32:46 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.exte

In [79]:
!dir crawler

 E 드라이브의 볼륨: UX200P
 볼륨 일련 번호: 1858-08E9

 E:\OneDrive - 둔전초등학교\01_과목별 수업자료\03_크롤링\03_차후 수업 추가할 것\scarappy_exam\crawler 디렉터리

2020-07-15  오후 10:32    <DIR>          .
2020-07-15  오후 10:32    <DIR>          ..
2020-07-15  오후 09:06    <DIR>          crawler
2020-07-15  오후 10:32             1,770 GmarketBestSellers.csv
2020-07-15  오후 09:06               257 scrapy.cfg
               2개 파일               2,027 바이트
               3개 디렉터리  144,591,007,744 바이트 남음


In [81]:
import pandas as pd
df = pd.read_csv('crawler/GmarketBestSellers.csv')
df.head(2)

,discount_rate,link,original_price,sales_price,title
0,0.0%,http://item.gmarket.co.kr/Item?goodscode=18401...,50000,50000,[한국투자증권] (한국투자증권) 온라인 금융상품권 5만원권
1,21.0%,http://item.gmarket.co.kr/Item?goodscode=15160...,10700,8500,[애경] 주방세제 순샘 대용량 3L x2개(라임1베리1)


In [88]:
!echo %cd%

E:\OneDrive - 둔전초등학교\01_과목별 수업자료\03_크롤링\03_차후 수업 추가할 것\scarappy_exam


In [90]:
df = df[['title','original_price', 'sales_price', 'discount_rate', 'link']]
df.head(3)

,title,original_price,sales_price,discount_rate,link
0,[한국투자증권] (한국투자증권) 온라인 금융상품권 5만원권,50000,50000,0.0%,http://item.gmarket.co.kr/Item?goodscode=18401...
1,[애경] 주방세제 순샘 대용량 3L x2개(라임1베리1),10700,8500,21.0%,http://item.gmarket.co.kr/Item?goodscode=15160...
2,루나샵 15일딱하루 5900원~티셔츠/원피스/박스티/롱티,19600,5900,70.0%,http://item.gmarket.co.kr/Item?goodscode=17254...


### 파이프라인스
- item을 출력하기 전에 실행되는 코드를 정의
- item을 원하는 형태로 처리 (메일 전송, 디비 저장등)
- 작성후 settings.py 에 등록

In [92]:
!type crawler\crawler\pipelines.py

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
from itemadapter import ItemAdapter


class CrawlerPipeline:
    def process_item(self, item, spider):
        return item


In [105]:
%%writefile crawler/crawler/pipelines.py
class CrawlerPipeline:
    
    # item-크롤링한 item
    # return item은 최종 프린트되는 곳으로 넘겨준다.
    def process_item(self, item, spider):
        print('='*50)
        print(item['sales_price'])
        print('='*50)
        if int(item['sales_price'] ) <= 50000: #50000이하만 저장
            with open('high_price.txt', 'at') as f: # append 모드로.
                f.write(item['title']+" - "+item['sales_price']+'\n')
        return item

Overwriting crawler/crawler/pipelines.py


- settings.py에 pipeline 설정
```
ITEM_PIPELINES = {
    'crawler.pipelines.CrawlerPipeline':300,
}
```
300 => 파이프라인 실행순서 (1 ~ 1000 사이숫자). 파이프라인이 여러개일때 실행 순서

In [106]:
!run.bat


(base) E:\OneDrive - 둔전초등학교\01_과목별 수업자료\03_크롤링\03_차후 수업 추가할 것\scarappy_exam>cd crawler 

(base) E:\OneDrive - 둔전초등학교\01_과목별 수업자료\03_크롤링\03_차후 수업 추가할 것\scarappy_exam\crawler>scrapy crawl GmarketBestSellers -o GmarketBestSellers.csv 
50000
5900
8500
6900

2020-07-15 22:58:56 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: crawler)
2020-07-15 22:58:56 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Windows-10-10.0.18362-SP0
2020-07-15 22:58:56 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-07-15 22:58:56 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'crawler',
 'NEWSPIDER_MODULE': 'crawler.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['crawler.spiders']}
2020-07-15 22:58:56 [scrapy.extensions.telnet] INFO: Telnet Password: cb8916b686a882d2
2020-07-15 22:58:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.exte


59900
41710
109000
14900
7900
14900


 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']
2020-07-15 22:58:56 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2020-07-15 22:58:56 [scrapy.middleware] INFO: Enabled item pipelines:
['crawler.pipelines.CrawlerPipeline']
2020-07-15 22:58:56 [scrapy.core.engine] INFO: Spider opened
2020-07-15 22:58:56 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2020-07-15 22:58:56 [scrapy.extensions.teln